## 1. Download dei dati ##

In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
# Download dataset
#!wget https://www.zemris.fer.hr/projects/LicensePlates/english/baza_slika.zip
# Unzip file zip
#!unzip -o -j baza_slika.zip "*.jpg" -d dataset

## 2. Preparazione dei dati ##

In [ ]:
import os

import gc
import matplotlib.pyplot as plt
import numpy as np

from PIL import Image
from skimage.transform import rescale
from skimage.util import view_as_windows
from torch.utils.data import DataLoader, random_split

In [ ]:
# Percorso alla cartella che contiene le immagini
datasetPath = "dataset"

# Lista per salvare le immagini caricate
imgList = []

# Scorrimento di tutti i file nella cartella
i = 0
for fileName in os.listdir(datasetPath):
    if fileName.lower().endswith(".jpg"):
        filePath = os.path.join(datasetPath, fileName)
        img = np.array(Image.open(filePath)).astype(np.float32) / 255.0
        img = rescale(img, (1/1.66, 1/1.66, 1)) # Passaggio da 640x480 a 386x289
        imgList.append(img)

In [ ]:
# Ridimensioniamo i blocchi
def resize_blocks(blocks, scaleY, scaleX):
    """
    Ridimensiona un insieme di blocchi in base a un fattore di scala
    blocks: (n_rows, n_cols, h, w, 3)
    scaleY: fattore di scala sull'asse Y
    scaleX: fattore di scala sull'asse X
    """
    blocksRes = []
    for i in range(blocks.shape[0]):
        row = []
        for j in range(blocks.shape[1]):
            blockLow = rescale(blocks[i][j], (scaleY,scaleX,1))
            blockUp = rescale(blockLow, (1//scaleY, 1//scaleY, 1), order=1)
            row.append(blockUp)
        blocksRes.append(row)
    return np.array(blocksRes)

# Ricostruiamo l'immagine con i nuovi blocchi
def reconstruct_img(blocks, stepY, stepX):
    """
    Ricostruisce un'immagine RGB da blocchi ridimensionati sovrapposti.
    Applica media nelle regioni sovrapposte.
    blocks: (n_rows, n_cols, h, w, 3)
    stepY: distanza verticale fra blocchi
    stepX: distanza orizzontale fra blocchi
    """
    nRows, nCols, h, w, c = blocks.shape
    H = (nRows - 1) * stepY + h
    W = (nCols - 1) * stepX + w
    canvas = np.zeros((H, W, c), dtype=np.float64)
    weight = np.zeros((H, W, c), dtype=np.float64)
    for i in range(nRows):
        for j in range(nCols):
            top = i * stepY
            left = j * stepX
            canvas[top:top+h, left:left+w, :] += blocks[i, j]
            weight[top:top+h, left:left+w, :] += 1
    return (canvas / np.maximum(weight, 1)).astype(np.float64)

def downscale_data(img, windowShape, scale, step):
    blocks = view_as_windows(img, windowShape, step).squeeze()
    blocksLow = resize_blocks(blocks, scale, scale)
    imgLow = reconstruct_img(blocksLow, step, step)
    return imgLow

In [ ]:
from torch.utils.data import Dataset
import torch
from torchvision import transforms
import numpy as np

class ImgListDataset(Dataset):
    def __init__(self, imgList):
        self.imgList = imgList
        self.target_size = (289, 386)  # Altezza, larghezza desiderata
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(self.target_size),
            transforms.ToTensor()
        ])

    def __len__(self):
        return len(self.imgList)

    def __getitem__(self, idx):
        img = self.imgList[idx]
        if isinstance(img, np.ndarray):
            img = self.transform(img)
        return img



In [ ]:
# Divisione dell'intero dataset in training set, validation set e test set (Circa 80%, 10% e 10%)
trainSet, valSet, testSet = random_split(imgList, [0.8, 0.1, 0.1])

windowShape = (96,96,3)
scale = 0.5
step = 24

# Creazione dei dataset di immagini ricostruite (downscale-upscale)
trainSetRic = []
valSetRic = []
testSetRic = []

for el in trainSet:
    imgLow = downscale_data(el, windowShape, scale, step)
    trainSetRic.append(imgLow)

for el in valSet:
    imgLow = downscale_data(el, windowShape, scale, step)
    valSetRic.append(imgLow)

for el in testSet:
    imgLow = downscale_data(el, windowShape, scale, step)
    testSetRic.append(imgLow)

In [ ]:
# Creazione dei DataLoader per i set
batchSize = 6
numWorkers = 2

# Dataloader per le immagini originali
trainSetDataset = ImgListDataset(trainSet)
valSetDataset = ImgListDataset(valSet)
testSetDataset = ImgListDataset(testSet)

#TODO Controllare bene se usare shuffle=True
trainDataload = DataLoader(trainSetDataset, batch_size=batchSize, shuffle=True, num_workers=numWorkers)
valDataload = DataLoader(valSetDataset, batch_size=batchSize, num_workers=numWorkers)
testDataload = DataLoader(testSetDataset, batch_size=batchSize, num_workers=numWorkers)

# Dataloader per le immagini originali
trainSetRicDataset = ImgListDataset(trainSetRic)
valSetRicDataset = ImgListDataset(valSetRic)
testSetRicDataset = ImgListDataset(testSetRic)

trainDataloadRic = DataLoader(trainSetRicDataset, batch_size=batchSize, shuffle=True, num_workers=numWorkers)
valDataloadRic = DataLoader(valSetRicDataset, batch_size=batchSize, num_workers=numWorkers)
testDataloadRic = DataLoader(testSetRicDataset, batch_size=batchSize, num_workers=numWorkers)

print("Effettuata suddivisione:")
print(f"- Training-set: {len(trainSet)} campioni.")
print(f"- Validation-set: {len(valSet)} campioni.")
print(f"- Test-set: {len(testSet)} campioni.")
print(f"- Training-set Ric: {len(trainSetRic)} campioni.")
print(f"- Validation-set Ric: {len(valSetRic)} campioni.")
print(f"- Test-set Ric: {len(testSetRic)} campioni.")

Fin qui tutto va bene...

## 3. Architettura ##

In [ ]:
import torch
import torch.nn as nn

class DnCNN(nn.Module):
    def __init__(self, in_channels=3, out_channels=64, kernel_size=3, padding=1):
        super(DnCNN, self).__init__()
        layers = []
            # Primo blocco convoluzionale
            # in_channel = 3 poichè 3 canali
            # out_channel = 64 come scritto nella traccia (num feat)
            # kernel_size = 3 come scritto nella traccia (dim spaziale 3x3)

        # Iter 1: Convolution + ReLU
        layers.append(nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, padding=padding))
        layers.append(nn.ReLU())

        # Iters 2-16: Convolution + BatchNorm + ReLU
        for i in range(15):
            layers.append(nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=kernel_size, padding=padding))
            layers.append(nn.BatchNorm2d(out_channels))                    # Corrisponde al numero di canali di output del livello precedente
            layers.append(nn.ReLU())

        # Iters 17: Convolution
        layers.append(nn.Conv2d(in_channels=out_channels, out_channels=in_channels, kernel_size=kernel_size, padding=padding))

        # *layers corrisponde a layers[0],layers[1],..., quindi già spalmati come singoli elementi
        self.features = nn.Sequential(*layers)

        self.apply(self.kernel_initializer)

    def forward(self, x):
        return self.features(x)

    # Equivalente a kernel_inizializer="Orthogonal" in pytorch
    def kernel_initializer(self, module):
        if isinstance(module, nn.Conv2d):
            nn.init.orthogonal_(module.weight)


## 4. Addestramento ##

In [ ]:
# Librerie
import time

In [ ]:
# Abilitiamo il dispositivo GPU per il training
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# Criterion deve essere torch.nn.MSELoss(size_average=None, reduce=None, reduction='mean')
# Legenda dataloader:
#   - dlTrain = immagini originali training
#   - dlTrainRic = immagini ricostruite (downscalate-upscalate) training
#   - dlVal = immagini originali validation
#   - dlValRic = immagini ricostruite (downscalate-upscalate) validation


def training (dlTrain, dlTrainRic, dlVal, dlValRic, numEpoch, model, criterion, optimizer, bestMse, bestPsnr, bestEpoch, outputPath):

    # Liste dei risultati definite per la visualizzazione
    MSETrainList = []
    MSEValList = []
    PSNRTrainList = []
    PSNRValList = []

    # Iterazione per ogni epoch
    for epoch in range(numEpoch):

        model.cuda()
        # Conteggio del tempo per misurare la durata di un'epoca
        since = time.time()

        # Inizializzazione delle variabili
        modelMseTrain = 0.0
        totalSize = 0

        # Modello impostato in traning mode
        model.train()

        # Iterazione per ogni batch
        # Devo iterare su i 2 dataloader contemporaneamente, per fare ciò utilizzo zip
        for (inputsTrain, inputsRic) in zip(dlTrain, dlTrainRic):

            # Converto gli input in tensori float e li carico nella GPU
            inputsTrain = inputsTrain.type(torch.FloatTensor).cuda()
            inputsRic = inputsRic.type(torch.FloatTensor).cuda()

            # Reset dei gradienti, altrimenti i vecchi gradienti sono sommati ai nuovi, piuttosto
            # che essere sovrascritti
            optimizer.zero_grad()
            model.zero_grad()

            # Calcolo dettagli e ricostruzione immagine
            yTrain = model(inputsRic)
            y = yTrain + inputsRic

            # Calcolo della MSE
            loss = criterion(y, inputsTrain)
            # size(0) restituisce il numero di campioni nel batch, quindi si sta moltiplicando la loss
            # media per il numero di elementi per ottenere la somma totale della loss
            modelMseTrain += loss.item() * inputsTrain.size(0)
            totalSize += inputsTrain.size(0)

            # Calcolando il gradiente del tensore attuale
            loss.backward()
            # Esecuzione di un singolo step di ottimizzazione in modo tale da aggiornare i pesi
            optimizer.step()

            del loss

        # Calcolo della MSE medio e del PSNR medio  dell'epoch
        modelMseEpochTrain = modelMseTrain/totalSize
        modelPsnrEpochTrain = 10 * torch.log10(torch.tensor(1/modelMseEpochTrain))

        # Salvataggio dei pesi per ogni iterazione
        # torch.save(model.state_dict(), outputPath + "train_weights.pth")

        # Modello impostato in validation mode
        model.eval()

        # Inizializzazione delle variabili
        modelMseVal = 0.0
        totalSizeVal = 0

        with torch.no_grad():               # Disattiva il calcolo dei gradienti
        # Iterazione per ogni bach
        # Devo iterare su i 2 dataloader contemporaneamente, per fare ciò utilizzo zip
            for (inputsVal, inputsValRic) in zip(dlVal, dlValRic):

                # Converto gli input in tensori float e li carico nella GPU
                inputsVal = inputsVal.type(torch.FloatTensor).cuda()
                inputsValRic = inputsValRic.type(torch.FloatTensor).cuda()

                # Calcolo dettagli
                yVal = model(inputsValRic)
                y = yVal + inputsValRic

                # Calcolo della MSE
                loss = criterion(y, inputsVal)
                # size(0) restituisce il numero di campioni nel batch, quindi si sta moltiplicando la loss
                # media per il numero di elementi per ottenere la somma totale della loss
                modelMseVal += loss.item() * inputsVal.size(0)
                totalSizeVal += inputsVal.size(0)

            # Calcolo della MSE medio e del PSNR medio  dell'epoch
            modelMseEpochVal = modelMseVal/totalSizeVal
            modelPsnrEpochVal = 10 * torch.log10(torch.tensor(1/modelMseEpochVal))
            timeElapsed = time.time()-since

        print('[Epoch %d][Train on %d [MSE: %.4f  PSNR: %.4f]][Val on %d [MSE: %.4f  PSNR: %.4f]][Time: %.0f m %.0f s]'
                %(epoch, totalSize, modelMseEpochTrain, modelPsnrEpochTrain, totalSizeVal, modelMseEpochVal,
                modelPsnrEpochVal, timeElapsed // 60, timeElapsed % 60))

        # Salvaggio dei risultati migliori
        if (modelMseEpochVal < bestMse):
            print("-------Saving best weights-------")
            bestMse = modelMseEpochVal
            bestPsnr = modelPsnrEpochVal
            bestEpoch = epoch
            # Salvataggio dei migliori risultati
            try:
                torch.save(model.cpu().state_dict(), outputPath)
                print("-------Best weights saved-------")
            except Exception as e:
                print("Error:", e)

        # Salvataggio dei risultati per la visualizzazione
        MSETrainList.append(modelMseEpochTrain)
        MSEValList.append(modelMseEpochVal)
        PSNRTrainList.append(modelPsnrEpochTrain)
        PSNRValList.append(modelPsnrEpochVal)

    return bestMse, bestPsnr, MSETrainList, MSEValList, PSNRTrainList, PSNRValList




In [ ]:
# Funzione di testing

def testing (dlTest, dlTestRic, model, criterion, weightPath):

    # Caricamento dei pesi
    model.load_state_dict(torch.load(weightPath, map_location=torch.device("cpu")))
    model.cuda()


    # Conteggio del tempo per misurare la durata di un'epoca
    since = time.time()

    # Inizializzazione delle variabili
    modelMseTest = 0.0
    totalSize = 0
    outputsTest = []


    #model.load_state_dict(torch.load(weightPath , map_location=device))
    model.eval()

    # Iterazione su batch
    with torch.no_grad():               # Disattiva il calcolo dei gradienti
        for (inputsTest, inputsTestRic) in zip(dlTest, dlTestRic):


            # Converto gli input in tensori float e li carico nella GPU
            inputsTest = inputsTest.type(torch.FloatTensor).cuda()
            inputsTestRic = inputsTestRic.type(torch.FloatTensor).cuda()

            # Calcolo dettagli e ricostruzione immagine
            yTest = model(inputsTestRic)
            y = yTest + inputsTestRic
            outputsTest.append(y)

            # Calcolo della MSE
            loss = criterion(y, inputsTest)
            # size(0) restituisce il numero di campioni nel batch, quindi si sta moltiplicando la loss
            # media per il numero di elementi per ottenere la somma totale della loss
            modelMseTest += loss.item() * inputsTest.size(0)
            totalSize += inputsTest.size(0)


        # Calcolo della MSE medio e del PSNR medio  dell'epoch
        modelMseEpochTest = modelMseTest/totalSize
        modelPsnrEpochTest = 10 * torch.log10(torch.tensor(1/modelMseEpochTest))
        timeElapsed = time.time()-since

        print("[Test] [MSE: %.4f  PSNR: %.4f] [Time: %.0f m %.0f s]"
          %(modelMseEpochTest, modelPsnrEpochTest, timeElapsed // 60, timeElapsed % 60))

        return outputsTest

## 5. Valutazione delle prestazioni ##

In [ ]:
def graphVisualization(MseModelTrain, MseModelVal, PSNRModelTrain, PSNRModelVal):

    # Visualizzazione delle liste come subplot
    fig, ax = plt.subplots(2, 1, figsize=(4, 5), constrained_layout=True)

    # Visualizzazione di MSE Training vs Validation
    ax[0].set_xlabel("Epoch", fontsize=10)
    ax[0].set_ylabel("MSE", fontsize=10)
    ax[0].set_title("MSE Training vs Validation", fontsize=10)
    ax[0].plot(range(len(MseModelTrain)), MseModelTrain, color="r", marker="o", label="Training MSE")
    ax[0].plot(range(len(MseModelVal)), MseModelVal, color="b", marker="o", label="Validation MSE")
    ax[0].set_xlim([0, max(len(MseModelTrain), len(MseModelVal))-0.5])
    ax[0].set_ylim([0.02, 0.2])
    ax[0].legend()
    ax[0].grid(True)

    # Visualizzazione di PSNR Training vs Validation
    ax[1].set_xlabel("Epoch", fontsize=10)
    ax[1].set_ylabel("PSNR", fontsize=10)
    ax[1].set_title("PSNR Training vs Validation", fontsize=10)
    ax[1].plot(range(len(PSNRModelTrain)), PSNRModelTrain, color="r", marker="o", label="Training PSNR")
    ax[1].plot(range(len(PSNRModelVal)), PSNRModelVal, color="b", marker="o", label="Validation PSNR")
    ax[1].set_xlim([0, max(len(PSNRModelTrain), len(PSNRModelVal))-0.5])
    ax[1].set_ylim([50, 100])
    ax[1].legend()
    ax[1].grid(True)

    plt.show()



In [ ]:
# Iper parametri
#epochList = [20, 30, 40, 50, 60]
#learningRateList = [0.01, 0.001, 0.0001]

epochList = [20,30,40]
learningRateList = [0.01,0.001,0.0001]



In [ ]:
# Main
import torch.optim as optim

model = DnCNN().cuda()
criterion = torch.nn.MSELoss(size_average=None, reduce=None, reduction='mean')

bestMSEComb = 9000000000000000000.0              # Miglior MSE tra tutte le combinazioni
bestPSNRComb = -900000000000000000.0             # Miglior PSNR tra tutte le combinazioni

weightPath = "best_weights.pth"
weightPathComb = "best_weights_lr.pth"

In [ ]:

# Training
for numEpoch in epochList:
    for lr in learningRateList:
        optimizer = optim.Adam(model.parameters(), lr=lr)

        bestMse = 9000000000000000000.0             # Miglior MSE della singola combinazione
        bestPsnr = -900000000000000000.0            # Miglior PSNR della singola combinazione
        bestEpoch = -1

        bestMSES, bestPSNRS, MSETrainList, MSEValList, PSNRTrainList, PSNRValList = training (trainDataload, trainDataloadRic, valDataload, valDataloadRic, numEpoch, model, criterion, optimizer, bestMse, bestPsnr, bestEpoch, weightPath)

        # Aggiorno i valori di miglior MSE
        if bestMSES < bestMSEComb:
            bestMSEComb = bestMSES
            bestPSNRComb = bestPSNRS
            # Caricamento dei pesi
            torch.save(model.cpu().state_dict(), weightPathComb)
            print(f"------Best Combination saved [Epoch: {numEpoch} - Learning Rate: {lr}]-------")
            model.cuda()

        #graphVisualization(MSETrainList, MSEValList, PSNRTrainList, PSNRValList)

print(f"Best MSE: {bestMSEComb}")
print(f"Best PSNR: {bestPSNRComb}")





In [ ]:
# Testing
outputsTest = testing(testDataload, testDataloadRic, model, criterion, weightPathComb)

for i, batch in enumerate(outputsTest):
    for j in range(batch.size(0)):
        img = batch[j].detach().cpu()
        img = img.permute(1, 2, 0).numpy()  # [C, H, W] -> [H, W, C]

        plt.figure()
        plt.imshow(img)
        plt.title(f'Output Image {i}_{j}')
